In [1]:
!pip install pandas
!pip install pymongo
!pip install boto3

In [5]:
import pandas as pd
import numpy as np
from io import BytesIO, StringIO
from pymongo import MongoClient
import boto3
import glob

In [7]:
# Exportando dados em batch para o MinIO
arquivos = glob.glob("arquivos/arquivos-batch/*.csv")

# # Importa cada arquivo .csv
dataframes = []
for arquivo in arquivos:
    dataframes.append(pd.read_csv(arquivo))

# # Concatena todos os dataframes
df = pd.concat(dataframes)

In [10]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='pyUuyVY1RvV1zJ4w',
    aws_secret_access_key='gUxBmkTbVktZcxCCcALKfhNltUkIQyiT',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [12]:
csv_buffer = StringIO()
df.to_csv(csv_buffer)
client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-07', Key="batch/all.csv")

{'ResponseMetadata': {'RequestId': '17939FCAE7B93BCC',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Wed, 01 Nov 2023 22:19:39 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"daa1a77647f7d203250bc24f8cfe5eaf"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '17939FCAE7B93BCC',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"daa1a77647f7d203250bc24f8cfe5eaf"'}

In [13]:
obj = client.get_object(
    Bucket='aula-07', 
    Key="batch/all.csv"
).get("Body")

data = pd.read_csv(obj)

In [16]:
data.head()

,Unnamed: 0,id,nome,email,endereco,criado_em
0,0,58007732,Dr. Theo Sales,leonardosouza@example.net,Viela de Castro Brazil,2023-11-01 22:00:35
1,1,21528566,Julia Dias,lopesjoao@example.org,Lagoa de Nunes 96 Brazil,2023-11-01 22:00:35
2,2,56282406,Pedro Lucas da Rosa,nina42@example.org,Vale de Silveira 71 Brazil,2023-11-01 22:00:35
3,3,24637095,Marina Ramos,monteiroemilly@example.com,Lagoa de Carvalho 4 Brazil,2023-11-01 22:00:35
4,4,16852893,Maria Vitória Silva,davi-luizcardoso@example.org,Colônia Nascimento 76 Brazil,2023-11-01 22:00:35


In [17]:
# Exportando dados com diferencial para o MinIO

old = pd.read_csv('arquivos/arquivos-diferencial/usuarios_old.csv')
new = pd.read_csv('arquivos/arquivos-diferencial/usuarios_new.csv')

In [34]:
csv_buffer = StringIO()
old.to_csv(csv_buffer)
client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-07', Key="diferencial/usuarios.csv")

{'ResponseMetadata': {'RequestId': '1793A11654792384',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Wed, 01 Nov 2023 22:43:22 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"32529cf13d6015e0aa33fa3a6ab221bd"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '1793A11654792384',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"32529cf13d6015e0aa33fa3a6ab221bd"'}

In [37]:
obj = client.get_object(
    Bucket='aula-07', 
    Key="diferencial/usuarios.csv"
).get("Body")

usuarios_prod = pd.read_csv(obj)

In [38]:
len(usuarios_prod)

550

In [39]:
diferencial = new[len(usuarios_prod):]

In [43]:
usuarios_final = pd.concat([usuarios_prod, diferencial], ignore_index=True)

In [48]:
csv_buffer = StringIO()
usuarios_final.to_csv(csv_buffer)
client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-07', Key="diferencial/usuarios.csv")

{'ResponseMetadata': {'RequestId': '1793A35A60175620',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Wed, 01 Nov 2023 23:24:53 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"a860df42fc992ddb36bffb2dba7e79e8"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '1793A35A60175620',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"a860df42fc992ddb36bffb2dba7e79e8"'}

In [ ]:
# Exportando dados com kafka para o minio

def read_s3_csv(bucket_name, key):
    obj = client.get_object(
        Bucket=bucket_name, 
        Key=key
    ).get("Body")

    return pd.read_csv(obj)
    
# Cria um consumidor com o Kafka
consumer = KafkaConsumer(
    'aula07-usuarios',
     bootstrap_servers=['awari-kafka:9093'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))



for message in consumer:
    
    new_record = pd.DataFrame([message.value])

    try:
        df = read_s3_csv('aula-07', 'streaming/usuarios.csv')
    except:
        
        df = pd.DataFrame(columns=['id', 'nome', 'email', 'endereco', 'criado_em'])

        # Anexa o novo registro ao DataFrame existente
    df = pd.concat([df, new_record], ignore_index=True)
    
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)  
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-07', Key="streaming/usuarios.csv")  # Atualiza o arquivo CSV no S3